In [1]:
pip install zeyrek

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
from snowballstemmer import TurkishStemmer
import zeyrek
import re

In [4]:
tqdm.pandas()

In [6]:
#veri setini yükleme
df = pd.read_csv("emlakodev.csv")
texts = df['ilanaciklama'].dropna().tolist()
sentences = sum([sent_tokenize(t) for t in texts], [])
sentences[:20]

['ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARALI OLUP ÖNÜ KAPANMAZ MUHTEŞEM SAKIZ ADASI MANZARALI 2+1 , 1+1 BAHÇELİ VE TERASLI DAİRELERDEN OLUŞAN PROJEDE KAPALI OTOPARK SPOR SALONU 7/24 GÜVENLİK AKILLI VE KONTROL MAKENİZMALI EV KONTROL SİSTEMİ VE\xa0 SONSUZLUK HAVUZU SİZLERİ BEKLİYOR.',
 '4 BLOKTAN OLUŞAN PROJEDE\xa0 SINIRLI SAYIDA DAİRELERİMİZ KALMIŞTIR.',
 'DAİRELERİMİZ KONUMLARINA GÖRE FİYAT FARKLILIĞI GÖSTERMEKTEDİR.',
 'PROJEMİZ RESİNDENCE NİTELİĞİNDE OLUP HİSSE TAPULU OLARAK SATIŞ OLACAĞI İÇİN KREDİYE UYGUN DEĞİLDİR.',
 'AİDAT ÖDEMESİ AYLIK 2.500 TL DİR.',
 'LÜTFEN DETAYLI BİLGİ İÇİN ARAYINIZ.',
 'ÇEŞME ATİLLA EMLAK',
 'ÇEŞME DALYAN MAHALLESİNDE\xa0 DALYAN MARİNAYA,PLAJLARA,MARKETLERE,DOLMUŞLARA YAKIN\xa0 KALORİFERLİ\xa0 YAZ KIŞ OTURMAYA UYGUN 2 ODA 1 SALON\xa0 SIFIR SATILIK DAİRE\xa0 DETAYLI BİLGİ VE RANDEVU İÇİN LÜTFEN ARAYINIZ.',
 'ÇEŞME ATİLLA EMLAK',
 'ÇEŞME AYASARANDA MEVKİİNDE 712 RESİDENCE DA ÖZEL YAPIM SATILIK 2+1 DUBLEX DAİRERESIDENCE DAİREMİZ ÖZEL DİZAYN EDİLMİŞ OLUP,

In [7]:
#stopwords listesi alma

stop_words = set(stopwords.words('turkish'))
stop_words_list = list(stop_words)
print(stop_words_list[:40])

['sanki', 'da', 'niçin', 'birşey', 'biz', 'şu', 'hem', 'her', 'çünkü', 'ile', 'neden', 'nereye', 'nerede', 'ne', 'siz', 'birkaç', 'veya', 'tüm', 'yani', 'mu', 'defa', 'hepsi', 'nasıl', 'o', 'çok', 'acaba', 'niye', 'ki', 'mü', 'ya', 'ise', 'ama', 'mı', 'diye', 'kim', 'az', 'daha', 'kez', 'nerde', 've']


In [8]:
stop_words

{'acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'en',
 'eğer',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'ile',
 'ise',
 'için',
 'kez',
 'ki',
 'kim',
 'mu',
 'mü',
 'mı',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niye',
 'niçin',
 'o',
 'sanki',
 'siz',
 'tüm',
 've',
 'veya',
 'ya',
 'yani',
 'çok',
 'çünkü',
 'şey',
 'şu'}

In [9]:
# Türkçe NLP araçları
lemmatizer = zeyrek.MorphAnalyzer()
stemmer = TurkishStemmer()
stop_words = set(stopwords.words('turkish'))

In [10]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-zA-ZçÇğĞıİöÖşŞüÜ\s]', '', sentence)
    tokens = word_tokenize(sentence)
    filtered_tokens = [token for token in tokens if token.isalpha() and token not in stop_words and len(token) > 2]
    lemmatized_tokens = []
    for token in filtered_tokens:
        try:
            analysis = lemmatizer.analyze(token)
            if analysis and analysis[0]:
                lemma = analysis[0][0][1]
                if lemma != 'Unk':  # Unk olmayanları ekle
                    lemmatized_tokens.append(lemma)
        except Exception:
            pass  # Hata durumunda kelimeyi atla
    stemmed_tokens = [stemmer.stemWord(token) for token in filtered_tokens]
    return lemmatized_tokens, stemmed_tokens

In [11]:
#cğmleleri işleme
sentences = []
for t in tqdm(texts, desc="Extracting sentences"):
    sentences.extend(sent_tokenize(t))

print(f"\nSample sentences: {sentences[:2]}\n")
print(f"İlk ilan: {texts[0]}")
print(f"Cümle sayısı: {len(sentences)}") 

Extracting sentences: 100%|██████████████████████████████████████████████████████| 2776/2776 [00:00<00:00, 4765.64it/s]


Sample sentences: ['ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARALI OLUP ÖNÜ KAPANMAZ MUHTEŞEM SAKIZ ADASI MANZARALI 2+1 , 1+1 BAHÇELİ VE TERASLI DAİRELERDEN OLUŞAN PROJEDE KAPALI OTOPARK SPOR SALONU 7/24 GÜVENLİK AKILLI VE KONTROL MAKENİZMALI EV KONTROL SİSTEMİ VE\xa0 SONSUZLUK HAVUZU SİZLERİ BEKLİYOR.', '4 BLOKTAN OLUŞAN PROJEDE\xa0 SINIRLI SAYIDA DAİRELERİMİZ KALMIŞTIR.']

İlk ilan: ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARALI OLUP ÖNÜ KAPANMAZ MUHTEŞEM SAKIZ ADASI MANZARALI 2+1 , 1+1 BAHÇELİ VE TERASLI DAİRELERDEN OLUŞAN PROJEDE KAPALI OTOPARK SPOR SALONU 7/24 GÜVENLİK AKILLI VE KONTROL MAKENİZMALI EV KONTROL SİSTEMİ VE  SONSUZLUK HAVUZU SİZLERİ BEKLİYOR. 4 BLOKTAN OLUŞAN PROJEDE  SINIRLI SAYIDA DAİRELERİMİZ KALMIŞTIR. DAİRELERİMİZ KONUMLARINA GÖRE FİYAT FARKLILIĞI GÖSTERMEKTEDİR. PROJEMİZ RESİNDENCE NİTELİĞİNDE OLUP HİSSE TAPULU OLARAK SATIŞ OLACAĞI İÇİN KREDİYE UYGUN DEĞİLDİR. AİDAT ÖDEMESİ AYLIK 2.500 TL DİR. LÜTFEN DETAYLI BİLGİ İÇİN ARAYINIZ. ÇEŞME ATİLLA EMLAK
Cümle sayıs

In [12]:
print(f"\nSample sentences: {sentences[:2]}\n")


Sample sentences: ['ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARALI OLUP ÖNÜ KAPANMAZ MUHTEŞEM SAKIZ ADASI MANZARALI 2+1 , 1+1 BAHÇELİ VE TERASLI DAİRELERDEN OLUŞAN PROJEDE KAPALI OTOPARK SPOR SALONU 7/24 GÜVENLİK AKILLI VE KONTROL MAKENİZMALI EV KONTROL SİSTEMİ VE\xa0 SONSUZLUK HAVUZU SİZLERİ BEKLİYOR.', '4 BLOKTAN OLUŞAN PROJEDE\xa0 SINIRLI SAYIDA DAİRELERİMİZ KALMIŞTIR.']



In [13]:
import logging
logging.getLogger('zeyrek').setLevel(logging.CRITICAL)  # Sadece ciddi hataları göster
# Tokenization, lemmatization, and stemming
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []
batch_size = 1000
for i in range(0, len(sentences), batch_size):
    batch = sentences[i:i+batch_size]
    for sentence in tqdm(batch, desc=f"Processing batch {i//batch_size + 1}"):
        lemmatized, stemmed = preprocess_sentence(sentence)
        tokenized_corpus_lemmatized.append(lemmatized)
        tokenized_corpus_stemmed.append(stemmed)

print("\nLemmatized sample:", tokenized_corpus_lemmatized[:5])
print("Stemmed sample:", tokenized_corpus_stemmed[:5])

Processing batch 10: 100%|███████████████████████████████████████████████████████████| 451/451 [00:53<00:00,  8.46it/s]


Lemmatized sample: [['Çeşme', 'mevkii', 'Full', 'de', 'olmak', 'ön', 'kapanmak', 'muhteşem', 'Bahçe', 'daire', 'oluşmak', 'proje', 'otopark', 'spor', 'salon', 'Güvenlik', 'Akil', 'kontrol', 'kontrol', 'sistem', 'sonsuz', 'havuz', 'siz', 'beklemek'], ['blok', 'oluşmak', 'proje', 'sinirli', 'daire'], ['daire', 'göre', 'fiyat', 'göstermek'], ['proje', 'nitelik', 'olmak', 'his', 'tapu', 'olmak', 'kredi', 'uygun', 'değil'], ['aidat', 'ödemek']]
Stemmed sample: [['çeşme', 'ayasara', 'mevkiin', 'full', 'de', 'manzarali', 'olup', 'ön', 'kapanmaz', 'muhteşe', 'sakiz', 'adasi', 'manzarali', 'bahçel', 'terasli', 'daire', 'oluşa', 'proje', 'kapali', 'otopark', 'spor', 'salo', 'güvenlik', 'akilli', 'kontrol', 'makenizmal', 'kontrol', 'siste', 'sonsuzluk', 'havuz', 'siz', 'bekliyor'], ['blok', 'oluşa', 'proje', 'sinirli', 'sayi', 'daire', 'kalmiş'], ['daire', 'konum', 'gör', 'fiyat', 'farklilik', 'göstermek'], ['proje', 'resinde', 'nitelik', 'olup', 'his', 'tapul', 'olarak', 'satiş', 'olacaği', 'kr

In [14]:
#Çıktılardan veri seti oluşturma
lemmatized_df = pd.DataFrame({
    'original_sentence': sentences,
    'processed_tokens': [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]
})

stemmed_df = pd.DataFrame({
    'original_sentence': sentences,
    'processed_tokens': [' '.join(tokens) for tokens in tokenized_corpus_stemmed]
})

In [15]:
#CSV dosyası olarak kaydetme
lemmatized_df.to_csv(r"C:\Users\Elif\desktop\yapay_zeka_projesi\lemmatized_sentences.csv", index=False, encoding='utf-8-sig')
stemmed_df.to_csv(r"C:\Users\Elif\desktop\yapay_zeka_projesi\stemmed_sentences.csv", index=False, encoding='utf-8-sig')

print("\nFiles saved successfully:")
print("lemmatized_sentences.csv")
print("stemmed_sentences.csv")


Files saved successfully:
lemmatized_sentences.csv
stemmed_sentences.csv


In [16]:
for i in range(5):
    print(f"Cümle {i+1} - Base: {sentences[i]}")   #cumlelerın ılk hallerı-metınde goruldugu hallerı (ham verı)
    print(f"Cümle {i+1} - Lemmatized: {tokenized_corpus_lemmatized[i]}")  #cumlelerın lema hallerı 
    print(f"Cümle {i+1} - Stemmed: {tokenized_corpus_stemmed[i]}")  #cumlelerın Stem hallerı 
    print("\n")

Cümle 1 - Base: ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARALI OLUP ÖNÜ KAPANMAZ MUHTEŞEM SAKIZ ADASI MANZARALI 2+1 , 1+1 BAHÇELİ VE TERASLI DAİRELERDEN OLUŞAN PROJEDE KAPALI OTOPARK SPOR SALONU 7/24 GÜVENLİK AKILLI VE KONTROL MAKENİZMALI EV KONTROL SİSTEMİ VE  SONSUZLUK HAVUZU SİZLERİ BEKLİYOR.
Cümle 1 - Lemmatized: ['Çeşme', 'mevkii', 'Full', 'de', 'olmak', 'ön', 'kapanmak', 'muhteşem', 'Bahçe', 'daire', 'oluşmak', 'proje', 'otopark', 'spor', 'salon', 'Güvenlik', 'Akil', 'kontrol', 'kontrol', 'sistem', 'sonsuz', 'havuz', 'siz', 'beklemek']
Cümle 1 - Stemmed: ['çeşme', 'ayasara', 'mevkiin', 'full', 'de', 'manzarali', 'olup', 'ön', 'kapanmaz', 'muhteşe', 'sakiz', 'adasi', 'manzarali', 'bahçel', 'terasli', 'daire', 'oluşa', 'proje', 'kapali', 'otopark', 'spor', 'salo', 'güvenlik', 'akilli', 'kontrol', 'makenizmal', 'kontrol', 'siste', 'sonsuzluk', 'havuz', 'siz', 'bekliyor']


Cümle 2 - Base: 4 BLOKTAN OLUŞAN PROJEDE  SINIRLI SAYIDA DAİRELERİMİZ KALMIŞTIR.
Cümle 2 - Lemmatized: ['blok'

In [17]:
datalem = pd.read_csv("lemmatized_sentences.csv")
datalem.head(5)

,original_sentence,processed_tokens
0,ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARAL...,Çeşme mevkii Full de olmak ön kapanmak muhteşe...
1,4 BLOKTAN OLUŞAN PROJEDE SINIRLI SAYIDA DAİRE...,blok oluşmak proje sinirli daire
2,DAİRELERİMİZ KONUMLARINA GÖRE FİYAT FARKLILIĞI...,daire göre fiyat göstermek
3,PROJEMİZ RESİNDENCE NİTELİĞİNDE OLUP HİSSE TAP...,proje nitelik olmak his tapu olmak kredi uygun...
4,AİDAT ÖDEMESİ AYLIK 2.500 TL DİR.,aidat ödemek


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
#ön işlenmiş token listelerini tekrar metine çeviriyoruz
lemmatized_texts = [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]

lemmatized_texts[:3]

['Çeşme mevkii Full de olmak ön kapanmak muhteşem Bahçe daire oluşmak proje otopark spor salon Güvenlik Akil kontrol kontrol sistem sonsuz havuz siz beklemek',
 'blok oluşmak proje sinirli daire',
 'daire göre fiyat göstermek']

In [20]:
#tf-idf vektöritizeri başlatma
vectorizer = TfidfVectorizer()

#tf-idf vektörlerini dönüştürür.
tfidf_matrix = vectorizer.fit_transform(lemmatized_texts)

#tf-idf vektörleştirme işleminde kullanılan tüm kelimelerin listesini döndürür.
feature_names = vectorizer.get_feature_names_out()

#tf-idf matrisini pandas dataframeine dönüştürme
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

print(tfidf_df.head())

tfidf_df.to_csv("tfidf_lemmatized.csv", index=False)

   aassm  abartmak  abd  abdi  abdullah  abdurrahim  abdurrahman  abdülhamid  \
0    0.0       0.0  0.0   0.0       0.0         0.0          0.0         0.0   
1    0.0       0.0  0.0   0.0       0.0         0.0          0.0         0.0   
2    0.0       0.0  0.0   0.0       0.0         0.0          0.0         0.0   
3    0.0       0.0  0.0   0.0       0.0         0.0          0.0         0.0   
4    0.0       0.0  0.0   0.0       0.0         0.0          0.0         0.0   

   abdülhamit  abi  ...  şubat  şube  şule  şut  şömine  şöyle  şükraniye  \
0         0.0  0.0  ...    0.0   0.0   0.0  0.0     0.0    0.0        0.0   
1         0.0  0.0  ...    0.0   0.0   0.0  0.0     0.0    0.0        0.0   
2         0.0  0.0  ...    0.0   0.0   0.0  0.0     0.0    0.0        0.0   
3         0.0  0.0  ...    0.0   0.0   0.0  0.0     0.0    0.0        0.0   
4         0.0  0.0  ...    0.0   0.0   0.0  0.0     0.0    0.0        0.0   

   şükür  şüphe  şık  
0    0.0    0.0  0.0  
1    0.0  

In [21]:
#ilk cümlenin tf-idf skorlarını alma
first_sentence_vector = tfidf_df.iloc[0]

#skorları sıralama küçükten büyüğe
top_5_words = first_sentence_vector.sort_values(ascending=False).head(5)

print("İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:")
print(top_5_words)

İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:
kontrol     0.494849
sonsuz      0.319231
akil        0.262493
çeşme       0.250631
kapanmak    0.215430
Name: 0, dtype: float64


In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#kelime vektörlerine bakacağız
balkon_index = feature_names.tolist().index('balkon')

balkon_vector = tfidf_matrix[:, balkon_index].toarray()

tfidf_vectors = tfidf_matrix.toarray()

similarities = cosine_similarity(balkon_vector.T, tfidf_vectors.T)

similarities = similarities.flatten()
top_5_indices = similarities.argsort()[-6:][::-1]

for index in top_5_indices:
    print(f"{feature_names[index]}: {similarities[index]:.4f}")

balkon: 1.0000
cam: 0.3620
banyo: 0.3164
âdet: 0.2534
çift: 0.2401
geniş: 0.2065


In [23]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [24]:
import gensim
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [27]:
datastem = pd.read_csv("stemmed_sentences.csv")
datastem.head(5)

,original_sentence,processed_tokens
0,ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARAL...,çeşme ayasara mevkiin full de manzarali olup ö...
1,4 BLOKTAN OLUŞAN PROJEDE SINIRLI SAYIDA DAİRE...,blok oluşa proje sinirli sayi daire kalmiş
2,DAİRELERİMİZ KONUMLARINA GÖRE FİYAT FARKLILIĞI...,daire konum gör fiyat farklilik göstermek
3,PROJEMİZ RESİNDENCE NİTELİĞİNDE OLUP HİSSE TAP...,proje resinde nitelik olup his tapul olarak sa...
4,AİDAT ÖDEMESİ AYLIK 2.500 TL DİR.,aidat ödemes aylik dir


In [28]:
stemmed_texts = [' '.join(tokens) for tokens in tokenized_corpus_stemmed]

stemmed_texts[:3]

['çeşme ayasara mevkiin full de manzarali olup ön kapanmaz muhteşe sakiz adasi manzarali bahçel terasli daire oluşa proje kapali otopark spor salo güvenlik akilli kontrol makenizmal kontrol siste sonsuzluk havuz siz bekliyor',
 'blok oluşa proje sinirli sayi daire kalmiş',
 'daire konum gör fiyat farklilik göstermek']

In [29]:
#tf-idf vektöritizeri başlatma
vectorizer = TfidfVectorizer()

#tf-idf vektörlerini dönüştürür.
tfidf_matrix = vectorizer.fit_transform(stemmed_texts)

#tf-idf vektörleştirme işleminde kullanılan tüm kelimelerin listesini döndürür.
feature_names = vectorizer.get_feature_names_out()

#tf-idf matrisini pandas dataframeine dönüştürme
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

print(tfidf_df.head())
tfidf_df.to_csv("tfidf_stemmed.csv", index=False)

   aalaça  aass  aaç  aba  abartılmamış  abdde  abdi  abdiipekçi  \
0     0.0   0.0  0.0  0.0           0.0    0.0   0.0         0.0   
1     0.0   0.0  0.0  0.0           0.0    0.0   0.0         0.0   
2     0.0   0.0  0.0  0.0           0.0    0.0   0.0         0.0   
3     0.0   0.0  0.0  0.0           0.0    0.0   0.0         0.0   
4     0.0   0.0  0.0  0.0           0.0    0.0   0.0         0.0   

   abdulhamidha  abdullah  ...  şöm  şöminebarbeküjakuz  şöminel  şömineor  \
0           0.0       0.0  ...  0.0                 0.0      0.0       0.0   
1           0.0       0.0  ...  0.0                 0.0      0.0       0.0   
2           0.0       0.0  ...  0.0                 0.0      0.0       0.0   
3           0.0       0.0  ...  0.0                 0.0      0.0       0.0   
4           0.0       0.0  ...  0.0                 0.0      0.0       0.0   

   şömines  şükrani  şükrü  şüphe  şık  şıklık  
0      0.0      0.0    0.0    0.0  0.0     0.0  
1      0.0      0.0    0

In [31]:
#ilk cümlenin tf-idf skorlarını alma
first_sentence_vector = tfidf_df.iloc[0]

#skorları sıralama (küçükten büyüğe)
top_5_words = first_sentence_vector.sort_values(ascending=False).head(5)

print("İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:")
print(top_5_words)

İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:
kontrol       0.402656
manzarali     0.264930
sakiz         0.253460
makenizmal    0.253460
ayasara       0.242597
Name: 0, dtype: float64


In [32]:
#kelime vektörlerine bakacağız
kat_index = feature_names.tolist().index('kat')

kat_vector = tfidf_matrix[:, kat_index].toarray()

tfidf_vectors = tfidf_matrix.toarray()

similarities = cosine_similarity(kat_vector.T, tfidf_vectors.T)

similarities = similarities.flatten()
top_5_indices = similarities.argsort()[-6:][::-1]

for index in top_5_indices:
    print(f"{feature_names[index]}: {similarities[index]:.4f}")

kat: 1.0000
dair: 0.2258
ar: 0.1910
mülkiyetli: 0.1797
banyo: 0.1771
bina: 0.1610


In [33]:
datalem = pd.read_csv("lemmatized_sentences.csv")
datalem.head(5)

,original_sentence,processed_tokens
0,ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARAL...,Çeşme mevkii Full de olmak ön kapanmak muhteşe...
1,4 BLOKTAN OLUŞAN PROJEDE SINIRLI SAYIDA DAİRE...,blok oluşmak proje sinirli daire
2,DAİRELERİMİZ KONUMLARINA GÖRE FİYAT FARKLILIĞI...,daire göre fiyat göstermek
3,PROJEMİZ RESİNDENCE NİTELİĞİNDE OLUP HİSSE TAP...,proje nitelik olmak his tapu olmak kredi uygun...
4,AİDAT ÖDEMESİ AYLIK 2.500 TL DİR.,aidat ödemek


In [34]:
parameters = [
    {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 300},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]

def train_and_save_model(corpus, params, model_name):
    model = Word2Vec(corpus, vector_size=params['vector_size'], window=params['window'], min_count=1, sg=1 if params['model_type'] == 'skipgram' else 0)
    model.save(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']}.model")
    print(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']} model saved!")

for param in parameters:
    train_and_save_model(tokenized_corpus_lemmatized, param, "lemmatized_model")

for param in parameters:
    train_and_save_model(tokenized_corpus_stemmed, param, "stemmed_model")

lemmatized_model_cbow_window2_dim100 model saved!
lemmatized_model_skipgram_window2_dim100 model saved!
lemmatized_model_cbow_window4_dim100 model saved!
lemmatized_model_skipgram_window4_dim100 model saved!
lemmatized_model_cbow_window2_dim300 model saved!
lemmatized_model_skipgram_window2_dim300 model saved!
lemmatized_model_cbow_window4_dim300 model saved!
lemmatized_model_skipgram_window4_dim300 model saved!
stemmed_model_cbow_window2_dim100 model saved!
stemmed_model_skipgram_window2_dim100 model saved!
stemmed_model_cbow_window4_dim100 model saved!
stemmed_model_skipgram_window4_dim100 model saved!
stemmed_model_cbow_window2_dim300 model saved!
stemmed_model_skipgram_window2_dim300 model saved!
stemmed_model_cbow_window4_dim300 model saved!
stemmed_model_skipgram_window4_dim300 model saved!


In [35]:
from gensim.models import Word2Vec

In [36]:
model_1 = Word2Vec.load("lemmatized_model_cbow_window2_dim100.model")
model_2 = Word2Vec.load("stemmed_model_skipgram_window4_dim100.model")
model_3 = Word2Vec.load("lemmatized_model_skipgram_window2_dim300.model")

def print_similar_words(model, model_name):
    similarity = model.wv.most_similar('balkon', topn=3)
    print(f"\n{model_name} Modeli - 'balkon' ile En Benzer 3 Kelime:")
    for word, score in similarity:
        print(f"Kelime: {word}, Benzerlik Skoru: {score}")

print_similar_words(model_1, "Lemmatized CBOW Window 2 Dim 100")
print_similar_words(model_2, "Stemmed Skipgram Window 4 Dim 100")
print_similar_words(model_3, "Lemmatized Skipgram Window 2 Dim 300")


Lemmatized CBOW Window 2 Dim 100 Modeli - 'balkon' ile En Benzer 3 Kelime:
Kelime: cam, Benzerlik Skoru: 0.9915151596069336
Kelime: tuvalet, Benzerlik Skoru: 0.9901226162910461
Kelime: teras, Benzerlik Skoru: 0.9874627590179443

Stemmed Skipgram Window 4 Dim 100 Modeli - 'balkon' ile En Benzer 3 Kelime:
Kelime: teras, Benzerlik Skoru: 0.9620236158370972
Kelime: kapatil, Benzerlik Skoru: 0.9579077363014221
Kelime: odalari, Benzerlik Skoru: 0.9566958546638489

Lemmatized Skipgram Window 2 Dim 300 Modeli - 'balkon' ile En Benzer 3 Kelime:
Kelime: tuvalet, Benzerlik Skoru: 0.9535260200500488
Kelime: teras, Benzerlik Skoru: 0.9505287408828735
Kelime: kiler, Benzerlik Skoru: 0.9496369957923889


In [37]:
datastem = pd.read_csv("stemmed_sentences.csv")
datastem.head(5)

,original_sentence,processed_tokens
0,ÇEŞME AYASARANDA MEVKİİNDE FULL DENİZ MANZARAL...,çeşme ayasara mevkiin full de manzarali olup ö...
1,4 BLOKTAN OLUŞAN PROJEDE SINIRLI SAYIDA DAİRE...,blok oluşa proje sinirli sayi daire kalmiş
2,DAİRELERİMİZ KONUMLARINA GÖRE FİYAT FARKLILIĞI...,daire konum gör fiyat farklilik göstermek
3,PROJEMİZ RESİNDENCE NİTELİĞİNDE OLUP HİSSE TAP...,proje resinde nitelik olup his tapul olarak sa...
4,AİDAT ÖDEMESİ AYLIK 2.500 TL DİR.,aidat ödemes aylik dir


In [38]:
parameters = [
    {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 300},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]

def train_and_save_model(corpus, params, model_name):
    model = Word2Vec(corpus, vector_size=params['vector_size'], window=params['window'], min_count=1, sg=1 if params['model_type'] == 'skipgram' else 0)
    model.save(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']}.model")
    print(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']} model saved!")

for param in parameters:
    train_and_save_model(tokenized_corpus_lemmatized, param, "lemmatized_model")

for param in parameters:
    train_and_save_model(tokenized_corpus_stemmed, param, "stemmed_model")

lemmatized_model_cbow_window2_dim100 model saved!
lemmatized_model_skipgram_window2_dim100 model saved!
lemmatized_model_cbow_window4_dim100 model saved!
lemmatized_model_skipgram_window4_dim100 model saved!
lemmatized_model_cbow_window2_dim300 model saved!
lemmatized_model_skipgram_window2_dim300 model saved!
lemmatized_model_cbow_window4_dim300 model saved!
lemmatized_model_skipgram_window4_dim300 model saved!
stemmed_model_cbow_window2_dim100 model saved!
stemmed_model_skipgram_window2_dim100 model saved!
stemmed_model_cbow_window4_dim100 model saved!
stemmed_model_skipgram_window4_dim100 model saved!
stemmed_model_cbow_window2_dim300 model saved!
stemmed_model_skipgram_window2_dim300 model saved!
stemmed_model_cbow_window4_dim300 model saved!
stemmed_model_skipgram_window4_dim300 model saved!


In [39]:
model_1 = Word2Vec.load("lemmatized_model_cbow_window2_dim100.model")
model_2 = Word2Vec.load("stemmed_model_skipgram_window4_dim100.model")
model_3 = Word2Vec.load("lemmatized_model_skipgram_window2_dim300.model")

def print_similar_words(model, model_name):
    similarity = model.wv.most_similar('kat', topn=3)
    print(f"\n{model_name} Modeli - 'kat' ile En Benzer 3 Kelime:")
    for word, score in similarity:
        print(f"Kelime: {word}, Benzerlik Skoru: {score}")

print_similar_words(model_1, "Lemmatized CBOW Window 2 Dim 100")
print_similar_words(model_2, "Stemmed Skipgram Window 4 Dim 100")
print_similar_words(model_3, "Lemmatized Skipgram Window 2 Dim 300")


Lemmatized CBOW Window 2 Dim 100 Modeli - 'kat' ile En Benzer 3 Kelime:
Kelime: bina, Benzerlik Skoru: 0.972588062286377
Kelime: iki, Benzerlik Skoru: 0.9691206812858582
Kelime: katil, Benzerlik Skoru: 0.9667516946792603

Stemmed Skipgram Window 4 Dim 100 Modeli - 'kat' ile En Benzer 3 Kelime:
Kelime: kati, Benzerlik Skoru: 0.8913896083831787
Kelime: sayis, Benzerlik Skoru: 0.8855994343757629
Kelime: yaşi, Benzerlik Skoru: 0.8834439516067505

Lemmatized Skipgram Window 2 Dim 300 Modeli - 'kat' ile En Benzer 3 Kelime:
Kelime: iki, Benzerlik Skoru: 0.9262877106666565
Kelime: katı, Benzerlik Skoru: 0.9251409769058228
Kelime: yaş, Benzerlik Skoru: 0.9088300466537476
